In [1]:
import shap
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from PIL import Image
import os
from matplotlib import pyplot as plt


# Custom activation functions
def relu(x):
    return tf.maximum(0.0, x)


def swish(x):
    return x * tf.sigmoid(x)


def elu(x, alpha=1.0):
    return tf.where(x >= 0.0, x, alpha * (tf.exp(x) - 1))


def softmax(x):
    return tf.exp(x) / tf.reduce_sum(tf.exp(x))


# Custom loss function
def categorical_Entropy_Loss_Function(y_true, y_pred):
    return -tf.reduce_mean(y_true * tf.math.log(y_pred))


# Custom accuracy function
def accuracy(y_true, y_pred):
    correct_predictions = tf.equal(tf.argmax(y_true, axis=1), tf.argmax(y_pred, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy


class ANNClassifier:
    def __init__(self, input_shape, num_classes, hidden_layers_sizes):
        """
        The constructor of the ANNClassifier class.
        :param input_shape: The shape of the input data
        :param num_classes: The number of classes
        :param hidden_layers_sizes: The sizes of the hidden layers
        """
        self.model = Sequential()
        self.model.add(Flatten(input_shape=input_shape))
        for i, layer_size in enumerate(hidden_layers_sizes):
            if i % 2 == 0:
                self.model.add(Dense(layer_size, activation=swish))
            else:
                self.model.add(Dense(layer_size, activation='elu'))
        self.model.add(Dense(num_classes, activation=softmax))

    def compile(self, optimizer):
        """
        Compiles the model.
        :param optimizer: The optimizer to use
        """
        self.model.compile(optimizer=optimizer, loss=categorical_Entropy_Loss_Function, metrics=[accuracy])

    def fit(self, X, y, epochs, batch_size, validation_data):
        """
        Fits the model to the training data.
        :param X: the training data
        :param y: the training labels
        :param epochs: the number of epochs
        :param batch_size: the batch size
        :param validation_data: the validation data and labels
        :return: the history of the training process
        """
        self.history = self.model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_data=validation_data)
        return self.history

    def explain(self, X):
        """
        Explains the model's predictions.
        :param X: the data used for prediction
        :return: the explanation
        """
        explainer = shap.DeepExplainer(self.model, X)
        return explainer.shap_values(X)

    def predict(self, X):
        """
        Predicts the labels of the data.
        :param X: the data used for prediction
        :return: the predicted labels
        """
        return self.model.predict(X)
    
    
    def SHAP_plot(self, X, y, index):
        """
        Plots the SHAP values.
        :param X: the data used for prediction
        :param y: the labels
        :param index: the index of the image to explain
        """
        explainer = shap.DeepExplainer(self.model, X)
        shap_values = explainer.shap_values(X)
        shap.image_plot(shap_values[index], X[index], show=False)
        plt.title('Predicted: ' + str(np.argmax(self.predict(X)[index])) + ' Actual: ' + str(np.argmax(y[index])))
        plt.show()
   

# Function to load images
def load_images(image_paths, target_size):
    images = []
    labels = []
    for image_path in image_paths:
        img = Image.open(image_path).convert('RGB')
        img = img.resize(target_size)
        img_array = np.array(img)
        images.append(img_array)
        label = os.path.basename(os.path.dirname(image_path))
        labels.append(label)
    images = np.array(images, dtype='float32') / 255.0
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)
    return images, labels


# Load and process images
image_paths = glob.glob('Car_Logo_Dataset/**/*.png', recursive=True)
X, y = load_images(image_paths, target_size=(32, 32))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ann = ANNClassifier(input_shape=(32, 32, 3), num_classes=10, hidden_layers_sizes=[128, 128, 128, 128, 128, 128, 128, 128, 128, 128])
ann.compile(optimizer='adam')
ann.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# SHAP plot
ann.SHAP_plot(X_test, y_test, 0)


2023-12-13 22:30:33.910422: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-13 22:30:33.910445: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-13 22:30:33.910451: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-13 22:30:33.910483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-13 22:30:33.910496: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


ValueError: in user code:

    File "/Users/zhaochengxin/anaconda3/envs/Learn/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/var/folders/fm/_6wfn0r96gjcpy7779d0j2fc0000gn/T/ipykernel_84314/242287152.py", line 34, in categorical_Entropy_Loss_Function  *
        return -tf.reduce_mean(y_true * tf.math.log(y_pred))

    ValueError: Dimensions must be equal, but are 32 and 10 for '{{node categorical_Entropy_Loss_Function/mul}} = Mul[T=DT_FLOAT](IteratorGetNext:1, categorical_Entropy_Loss_Function/Log)' with input shapes: [?,32], [?,10].
